In [1]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import re
import string
import pandas as pd
import nltk

In [2]:
analyzer = SentimentIntensityAnalyzer()
def sentiment_analyzer_scores(text):
    score = analyzer.polarity_scores(text)
    print(score)

In [3]:
linedata = []
with open("../../tmp3.csv") as f:
    for line in f:
        linedata.append(line.split(','))

In [4]:
linedata = pd.DataFrame(linedata)

In [5]:
linedata['text'] = linedata[6].fillna('') + linedata[7].fillna('') + linedata[8].fillna('') + linedata[9].fillna('') + linedata[10].fillna('') + linedata[11].fillna('')

In [6]:
linedata.rename(columns = {0:'id', 1:'user', 2:'time', 4:'lat', 5:'lon'}, inplace = True)

In [7]:
df = linedata[['id','user','time','lat','lon','text']]

In [22]:
df.head()

,id,user,time,lat,lon,text
0,'262619667330396161','Andrewthemark','2012-10-28 18:19:42',40.67360978,-73.69137707,'@yeahimjailed @marcushaight yeah but kristin ...
1,'262619681305792512','cabrera','2012-10-28 18:19:45',40.6879886,-73.9546139,'@zarabeatriz @dupre se viene Sandy.'\n
2,'262619684543811584','m0ff','2012-10-28 18:19:46',45.27374765,-75.86502686,'I wonder how bad my migraine is gonna be when...
3,'262619693376999424','Melbroadcasts','2012-10-28 18:19:48',40.77172216,-73.91315814,'Come on atleast save some of the wolves! http...
4,'262619694199107584','LeEmLeGiT','2012-10-28 18:19:48',41.11522062,-75.04597037,"'""@jellii19: Sandy #YouScared... Roll through ..."


In [17]:
df = df.dropna()

In [19]:
df['lat'] = df['lat'].str.split('[').apply(lambda x: x[1])

In [21]:
df['lon'] = df['lon'].str.split(']').apply(lambda x: x[0])

In [23]:
def prepocess(text):
    if 'via' in text:
        return re.match('(.*?)via.*?\s?(.*?)', re.sub(r"http\S+", "", re.sub(r"[^a-zA-Z0-9]+", ' ', re.sub(r'\d+', '', text.lower()).translate(str.maketrans('', '', string.punctuation)).strip()))).group(1)
    else:
        return re.sub(r"http\S+", "", re.sub(r"[^a-zA-Z0-9]+", ' ', re.sub(r'\d+', '', text.lower()).translate(str.maketrans('', '', string.punctuation)).strip()))

In [21]:
from pycorenlp import StanfordCoreNLP 

In [22]:
nlp = StanfordCoreNLP('http://localhost:9000')

In [31]:
# def get_sentiment(text):
#     res = nlp.annotate(text,
#                        properties={'annotators':'sentiment',
#             'outputFormat': 'json',
#             'timeout': 1000,
# })
#     print(text)
#     print('Sentiment:', res['sentences'][0]['sentiment'])
#     print('Sentiment score:', res['sentences'][0]['sentimentValue'])
#     print('Sentiment distribution (0-v. negative, 5-v.positive:', res['sentences'][0]['sentimentDistribution'])

In [145]:
def get_sentiment(text):
    res = nlp.annotate(text,
                       properties={'annotators':'sentiment',
            'outputFormat': 'json',
            'timeout': 1000,
})
    return(text,res['sentences'][0]['sentiment'],res['sentences'][0]['sentimentValue'],res['sentences'][0]['sentimentDistribution'])

In [154]:
a = []
for sent in tweet.tweet:
    a.append(get_sentiment(prepocess(sent)))

In [156]:
b = pd.DataFrame(a)

In [157]:
b.head()

,0,1,2,3
0,tsearby hurricane sandy blows up car check o...,Negative,1,"[0.09622443529724, 0.59071465495599, 0.2536024..."
1,east coast braces for b storm as hurricane san...,Negative,1,"[0.08524577521647, 0.65882100631568, 0.2236787..."
2,hurricane sandy deaths storm was cruel to new ...,Negative,1,"[0.10575421473174, 0.56868428783291, 0.2967027..."
3,outer banks begins clean up after hurricane sa...,Neutral,2,"[0.03949360792491, 0.28610957251427, 0.3991771..."
4,mitt romney refuses to talk about fema after h...,Negative,1,"[0.10489154985171, 0.54525896644645, 0.2463763..."
